In [1]:
from tqdm.notebook import tqdm
from IPython.display import display, HTML

import numpy as np
import pandas as pd

import plotly.graph_objects as go

import datetime
from pathlib import Path

import intake

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.utils import shuffle
import itertools

from tensorflow.keras.regularizers import l2
import tensorflow as tf
from tensorflow.keras import backend as K

from tensorflow.keras.layers import *

2024-05-27 16:21:11.289091: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-27 16:21:12.164803: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
catalog = intake.open_catalog(Path('../catalog.yml'))
source = getattr(catalog, 'treesat_multi')
df = source.read()

In [3]:
selected_bands = [f'B{x}' for x in range(2, 9)] + ['B8A', 'B11', 'B12', 'TCI_R', 'TCI_G', 'TCI_B']

In [4]:
target = source.metadata['categories']['multi'] # multi / trinary
labels = df[target].to_numpy()

label_year = df['YEAR'].astype(int).to_numpy()
# keep_indices = np.where(label_year >= 2017)[0]
keep_indices = np.where(label_year > 2010)[0]
label_year = np.where(label_year < 2017, 2017, label_year)

model_dir = Path('models').joinpath('seasons')
model_dir.mkdir(parents=True, exist_ok=True)

data_dir = Path('seasonal_median')

In [5]:
def get_features(years, seasons, data_dir, label_year):
    selected_data = np.zeros((label_year.shape[0], len(seasons), 6, 6, 13))
    for y, year in enumerate(years):
        for s, season in enumerate(seasons):
            filepath = data_dir.joinpath(f'processed_treesat_{year}{str(season).zfill(2)}.npy')
            with open(filepath, 'rb') as f:
                data = np.load(f)

            year_indices = np.where(label_year == year)
            selected_data[year_indices, s, ...] = data[year_indices]
    return selected_data

In [6]:
class KerasModelCreator:
    def normalise_X(self, X, p=1):
        upper = np.percentile(X, 100-p)
        lower = np.percentile(X, p)
    
        bounded_X = np.where(X > upper, np.median(X), X)
        bounded_X = np.where(X < lower, np.median(X), bounded_X)
        
        scaled_X = (bounded_X - lower)/(upper - lower)
        return scaled_X

    def run(self, X, y, model_path, epochs, batch_size=10, overwrite=False):
        ''' 
        If not overwrite and there's an existing model, the model will 
        continue trainingw if the given epoch is bigger than the previous,
        else just evaluate.
        Ensure train splits are the same across continuations / evaluations
        by not modifying the random_state in split_and_normalise.
        '''
        model_savepoint = model_path.parent.joinpath(model_path.stem)
        log_file = model_path.with_suffix('.log')

        if overwrite:
            for f in [model_path, log_file] + list(model_savepoint.glob('*')):
                f.unlink(missing_ok=True)

        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.1, random_state=1)

        for i in range(X.shape[-1]):
            X_train[...,i] = self.normalise_X(X_train[...,i])
            X_test[...,i] = self.normalise_X(X_test[...,i])
        
        default_metrics = ['accuracy', 'root_mean_squared_error', 'r2_score']
        
        if model_path.is_file():
            model = tf.keras.models.load_model(model_path)
        else:
            model = self.build_model(y, X.shape, default_metrics)

        callbacks = [
            tf.keras.callbacks.BackupAndRestore(
                model_savepoint, save_freq='epoch', delete_checkpoint=False
            ),
            tf.keras.callbacks.CSVLogger(log_file, append=True),
        ]

        if log_file.is_file():
            val_metrics = ['val_' + x for x in default_metrics]
            df = pd.read_csv(log_file)[['epoch', 'loss'] + default_metrics + val_metrics]
            df['epoch'] += 1
            print('Previous training:')
                
            display(HTML(df.to_html(index=False)))
        
        model.fit(
            X_train, y_train, epochs=epochs, verbose=1, batch_size=batch_size, callbacks=callbacks, 
            shuffle=True, validation_data=(X_test, y_test)
        )

        model.save(model_path)
        
    def build_model(self, y_train, input_shape, metrics):
        m = tf.keras.Sequential()

        m.add(Input(input_shape[1:]))
        
        m.add(Conv3D(
            filters=8, kernel_size=2, strides=1,
            padding='same', activation='relu'
        ))

        m.add(Flatten())
        
        m.add(Dense(
            y_train.shape[1], 
            activation='softmax', 
        ))
        m.compile(optimizer='rmsprop', loss='mse', metrics=metrics)
        return m

In [7]:
train_years = [2017, 2018, 2019, 2020]
all_seasons = [3, 6, 9, 12]

In [8]:
model_name = f'conv_year_seasons_{"_".join(map(str, all_seasons))}_years_{"_".join(map(str, train_years))}.keras'
model_path = model_dir.joinpath(model_name)

train_features = get_features(train_years, all_seasons, data_dir, label_year)

In [9]:
KerasModelCreator().run(
    train_features[keep_indices, ...], labels[keep_indices, ...],
    model_path, epochs=10, batch_size=10, overwrite=True
)

Epoch 1/10
4535/4535 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step - accuracy: 0.4135 - loss: 0.0391 - r2_score: 0.1006 - root_mean_squared_error: 0.1975 - val_accuracy: 0.5261 - val_loss: 0.0328 - val_r2_score: 0.2171 - val_root_mean_squared_error: 0.1812
Epoch 2/10
4535/4535 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - accuracy: 0.5423 - loss: 0.0316 - r2_score: 0.2279 - root_mean_squared_error: 0.1777 - val_accuracy: 0.5862 - val_loss: 0.0294 - val_r2_score: 0.2685 - val_root_mean_squared_error: 0.1714
Epoch 3/10
4535/4535 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - accuracy: 0.5666 - loss: 0.0296 - r2_score: 0.2597 - root_mean_squared_error: 0.1722 - val_accuracy: 0.5868 - val_loss: 0.0285 - val_r2_score: 0.2835 - val_root_mean_squared_error: 0.1688
Epoch 4/10
4535/4535 ━━━━━━━━━━━━━━━━━━━━ 6s 1ms/step - accuracy: 0.5854 - loss: 0.0285 - r2_score: 0.2387 - root_mean_squared_error: 0.1688 - val_accuracy: 0.5765 - val_loss: 0.0298 - val_r2_score: 0.2573 - val_root_mean_squared_error: 0.1727
Epoch 5/10
4535/4535 ━━━

In [16]:
year_shift = 0
test_years = [y + year_shift for y in train_years]
test_features = get_features(test_years, all_seasons, data_dir, label_year + year_shift)

test_features, test_labels = test_features[keep_indices, ...], labels[keep_indices, ...]
_, test_features, _, test_labels = train_test_split(
    test_features, test_labels, test_size=0.1, random_state=42)

for i in range(test_features.shape[-1]):
    test_features[...,i] = KerasModelCreator().normalise_X(test_features[...,i])
    
model = tf.keras.models.load_model(model_path)
model.evaluate(test_features, test_labels, verbose=0, return_dict=True)

{'accuracy': 0.6255209445953369,
 'loss': 0.026704134419560432,
 'r2_score': 0.3101809322834015,
 'root_mean_squared_error': 0.16341400146484375}

In [11]:
tf.keras.models.load_model(model_path).summary(expand_nested=True)

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv3d (Conv3D)                 │ (None, 4, 6, 6, 8)     │           840 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 1152)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 15)             │        17,295 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 36,272 (141.69 KB)

 Trainable params: 18,135 (70.84 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 18,137 (70.85 KB)

In [12]:
# import subprocess
# subprocess.run(['sudo', 'shutdown', 'now'])

In [13]:
# season_combinations = itertools.chain.from_iterable(
#     itertools.combinations(all_seasons, r) for r in range(1, len(all_seasons)+1))
# season_combinations = list(season_combinations)